The goal for this piece of code is:
- Clean the data for the following datasets:
    - K2
    - APOKSAC
    - APOGEE
    - GALAH

After we have cleaned the datas is
- Use K2 model to predict the mass for APOGEE and GALAH
- Use APOKSAC model to predict mass for APOGEE

The goal is to get a table contain the following for all the data sets and use this to predict the mass of stars.
The table include parameters such as:

- Effective Temperature
- Log g
- Iron Abundance [Fe/H]
- Alpha Abundance [alpha/Fe]
- Carbon Abundance [C/Fe]
- Nitrogen Abundance [N/Fe]
- Oxygen Abundance [O/Fe]

In addition, we will extract the stars id to allow for identification

- `tmass_id` 2MASS IDs
- `gaiadr3_source_id` Gaia Id

Importing the required packages

In [3]:
from astropy.table import Table
import pandas as pd         # For table/data management
from astroquery.utils.tap.core import TapPlus
from astroquery.gaia import Gaia

# Cleaning the GALAH Data

In [6]:
# Import the data and change into a data frame
def importGalah():
    galah = Table.read("RawData/galah_dr4_allspec_220713.fits", format = "fits")
    names = [name for name in galah.colnames if len(galah[name].shape) <= 1]
    galah = galah[names].to_pandas()
    return galah
galah = importGalah()

For Apogee Data, there are a few recommendations (this usually indicates that the data has some issues, e.g. unreliably broadening and other issues) for the flag that should be applied:
- `snr_c3_iraf > 30`
- `flag_sp == 0`
- For any element `X`, the flag `flag_X_fe == 0`

More information can be found [the GALAH Survey Website](https://www.galah-survey.org/dr3/flags/)

In [7]:
# Filter the data
# galah = galah.loc[galah["snr_c3_iraf"] > 30]
galah = galah.loc[galah["flag_sp"] == 0]
galah = galah.loc[galah["flag_fe_h"] == 0]
galah = galah.loc[galah["flag_al_fe"] == 0]
galah = galah.loc[galah["flag_c_fe"] == 0]
galah = galah.loc[galah["flag_n_fe"] == 0]
galah = galah.loc[galah["flag_o_fe"] == 0]
galah = galah.loc[galah["flag_li_fe"] == 0]

In [8]:
galah

,sobject_id,tmass_id,gaiadr3_source_id,ra,dec,flag_sp,chi2_sp,model_name,teff,e_teff,...,rv_k_is,ew_dib5780,sigma_dib5780,rv_dib5780,ew_dib5797,sigma_dib5797,rv_dib5797,ew_dib6613,sigma_dib6613,rv_dib6613
5,131216001101010,b'05225435-5857055',4762753422822077184,80.726471,-58.951542,0,0.801761,b'4750_2.50_-0.25',4747.277832,11.832772,...,85.141571,0.023134,0.252208,26.350906,0.018000,0.101814,-39.829533,0.029444,1.492405,31.226997
8,131216001101014,b'05242161-5901319',4762739782006427648,81.090057,-59.025547,0,0.866050,b'5000_4.50_0.00',5007.801270,9.640579,...,-56.604633,0.023699,0.536554,-5.562334,0.025659,0.737862,-5.556193,0.015717,0.446130,-5.526703
11,131216001101017,b'05221820-5902598',4762730882833911808,80.575874,-59.049953,0,0.843927,b'4750_4.50_0.00',4801.387207,8.895825,...,167.882294,-0.012029,0.074605,39.358860,0.016823,0.127976,-15.108765,0.005813,0.160391,-8.338925
16,131216001101022,b'05222284-5903420',4762727927896411392,80.595207,-59.061672,0,1.240397,b'5250_4.50_0.25',5278.634277,4.595638,...,-13.605973,0.015943,0.222795,30.253696,-0.010231,0.392958,-41.382511,0.007021,0.049410,11.469566
18,131216001101024,b'05225971-5905550',4762726897104253824,80.748810,-59.098625,0,0.875987,b'4500_4.00_0.00',4613.555176,14.729848,...,-133.567993,0.021436,0.170836,89.049568,-0.026330,0.095232,41.361435,0.009535,0.044633,11.150452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911431,220422002101020,b'08490304-4234488',5524145365131834880,132.262680,-42.580215,0,0.907719,b'4250_1.50_-0.50',4326.258789,6.605823,...,-27.567528,0.242692,0.821876,-4.200578,0.081907,0.482253,-31.767265,0.053212,0.267420,-4.404961
911484,220422002101081,b'08482763-4300045',5524113543714103168,132.115097,-43.001232,0,0.852804,b'5000_3.00_-0.25',4892.621094,7.416252,...,7.454720,0.013022,0.100351,42.018845,0.015768,0.125131,-49.141647,0.015822,0.718062,-11.059895
911566,220422002101199,b'08431567-4230079',5524898599314763136,130.815277,-42.502193,0,0.756809,b'5000_3.00_0.00',4921.517090,12.264063,...,27.553761,0.220546,1.030082,45.633423,0.062203,1.144449,13.689507,0.048689,0.384843,61.364906
911568,220422002101203,b'08433930-4230037',5524851114156313600,130.913773,-42.501072,0,0.790913,b'4500_2.50_0.00',4544.542969,24.683069,...,-41.585598,0.479357,0.681622,-23.666155,0.169192,0.646230,-36.374008,0.216087,0.376157,-15.404000


Now the appropriate flag has been used, we extract only the useful columns and save the data for later.

In [9]:
galah = galah[["tmass_id", "gaiadr3_source_id", "teff", "logg", "fe_h", "al_fe", "c_fe", "n_fe", "o_fe", "li_fe"]]
# Remove all rows with null values
galah.dropna()
galah.to_csv("ProcessedData/cleaned_galah.csv", index=False)

In [10]:
galah

,tmass_id,gaiadr3_source_id,teff,logg,fe_h,al_fe,c_fe,n_fe,o_fe,li_fe
5,b'05225435-5857055',4762753422822077184,4747.277832,2.502026,-0.295569,0.242638,0.114089,0.048963,0.213739,-0.925169
8,b'05242161-5901319',4762739782006427648,5007.801270,4.504993,0.065576,0.186914,-0.039235,0.194514,-0.380775,-0.658700
11,b'05221820-5902598',4762730882833911808,4801.387207,4.519791,-0.054487,-0.114537,-0.167340,-0.317684,-0.461599,-0.831258
16,b'05222284-5903420',4762727927896411392,5278.634277,4.480108,0.221903,0.212420,0.019690,0.207750,-0.092077,-0.576487
18,b'05225971-5905550',4762726897104253824,4613.555176,4.221701,-0.088724,0.339632,0.136008,-0.187697,-0.091664,-1.203047
...,...,...,...,...,...,...,...,...,...,...
911431,b'08490304-4234488',5524145365131834880,4326.258789,1.684273,-0.444126,0.440457,0.251971,0.182709,0.544992,-1.136488
911484,b'08482763-4300045',5524113543714103168,4892.621094,3.312382,-0.146132,0.454510,0.322544,0.366330,0.693901,-0.830099
911566,b'08431567-4230079',5524898599314763136,4921.517090,3.204505,0.015238,0.146001,0.029876,0.221036,0.257410,-1.011459
911568,b'08433930-4230037',5524851114156313600,4544.542969,2.545092,0.151254,0.061476,0.121679,-0.072001,0.024033,-1.081211


# Cleaning APOGEE Data

In [4]:
apogee = Table.read("RawData/allStar-dr17-synspec_rev1.fits", format="fits", hdu=1)
names = [name for name in apogee.colnames if len(apogee[name].shape) <= 1]
apogee = apogee[names].to_pandas()
galah

,FILE,APOGEE_ID,TARGET_ID,APSTAR_ID,ASPCAP_ID,TELESCOPE,LOCATION_ID,FIELD,ALT_ID,RA,...,CU_FE_ERR,CU_FE_FLAG,CE_FE,CE_FE_SPEC,CE_FE_ERR,CE_FE_FLAG,YB_FE,YB_FE_SPEC,YB_FE_ERR,YB_FE_FLAG
0,b'apStar-dr17-VESTA.fits',b'VESTA',b'apo1m.calibration.VESTA',b'apogee.apo1m.stars.calibration.VESTA',b'apogee.apo1m.synspec_fix.calibration.VESTA',b'apo1m',1,b'calibration',b' ',NaN,...,0.059403,2,NaN,NaN,NaN,64,NaN,NaN,1.0,2
1,b'apStar-dr17-2M00000002+7417074.fits',b'2M00000002+7417074',b'apo25m.120+12.2M00000002+7417074',b'apogee.apo25m.stars.120+12.2M00000002+7417074',b'apogee.apo25m.synspec_fix.120+12.2M00000002+...,b'apo25m',5046,b'120+12',b'none',0.000103,...,0.001221,2,NaN,NaN,NaN,64,NaN,NaN,1.0,2
2,b'apStar-dr17-2M00000019-1924498.fits',b'2M00000019-1924498',b'apo25m.060-75.2M00000019-1924498',b'apogee.apo25m.stars.060-75.2M00000019-1924498',b'apogee.apo25m.synspec_fix.060-75.2M00000019-...,b'apo25m',5071,b'060-75',b'none',0.000832,...,0.102594,2,NaN,NaN,NaN,64,NaN,NaN,1.0,2
3,b'apStar-dr17-2M00000032+5737103.fits',b'2M00000032+5737103',b'apo25m.116-04.2M00000032+5737103',b'apogee.apo25m.stars.116-04.2M00000032+5737103',b'apogee.apo25m.synspec_fix.116-04.2M00000032+...,b'apo25m',4424,b'116-04',b'none',0.001335,...,0.123839,2,NaN,NaN,NaN,64,NaN,NaN,1.0,2
4,b'apStar-dr17-2M00000032+5737103.fits',b'2M00000032+5737103',b'apo25m.N7789.2M00000032+5737103',b'apogee.apo25m.stars.N7789.2M00000032+5737103',b'apogee.apo25m.synspec_fix.N7789.2M00000032+5...,b'apo25m',4264,b'N7789',b'none',0.001335,...,0.107103,2,NaN,NaN,NaN,64,NaN,NaN,1.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733896,b'apStar-dr17-2M23595886-2009435.fits',b'2M23595886-2009435',b'apo25m.060-75.2M23595886-2009435',b'apogee.apo25m.stars.060-75.2M23595886-2009435',b'apogee.apo25m.synspec_fix.060-75.2M23595886-...,b'apo25m',5071,b'060-75',b'none',359.995258,...,NaN,258,NaN,NaN,NaN,64,NaN,NaN,NaN,2
733897,b'apStar-dr17-2M23595886+5726058.fits',b'2M23595886+5726058',b'apo25m.116-04.2M23595886+5726058',b'apogee.apo25m.stars.116-04.2M23595886+5726058',b'apogee.apo25m.synspec_fix.116-04.2M23595886+...,b'apo25m',4424,b'116-04',b'none',359.995265,...,NaN,259,-0.058750,-0.003090,0.072443,0,NaN,NaN,1.0,2
733898,b'apStar-dr17-2M23595921+5609479.fits',b'2M23595921+5609479',b'apo25m.NGC7789_MGA.2M23595921+5609479',b'apogee.apo25m.stars.NGC7789_MGA.2M23595921+5...,b'apogee.apo25m.synspec_fix.NGC7789_MGA.2M2359...,b'apo25m',-999,b'NGC7789_MGA',b'none',359.996744,...,NaN,259,-0.103446,-0.047786,0.097572,0,NaN,NaN,1.0,2
733899,b'asStar-dr17-2M23595949-7342592.fits',b'2M23595949-7342592',b'lco25m.SMC12.2M23595949-7342592',b'apogee.lco25m.stars.SMC12.2M23595949-7342592',b'apogee.lco25m.synspec_fix.SMC12.2M23595949-7...,b'lco25m',7218,b'SMC12',b'none',359.997887,...,0.174322,2,0.107710,0.163370,0.140188,0,NaN,NaN,1.0,2


For APOGEE, we do something similar by making sure all the parameters are working on are good.
More information can be found [SDSS explanantion of bitmasks](https://www.sdss.org/dr13/algorithms/bitmasks/#APOGEE_ASPCAPFLAG) and [SDSS recommendations for bitmasks](https://www.sdss.org/dr13/irspec/abundances/)

The parameters
- `ASPCAPFLAG & STAR_BAD == 0` (`STAR_BAD` is the 23rd bit (starting from 0), or  `STAR_BAD = 0b 1000 0000 0000 0000 0000 0000`)
    - `STAR_BAD` is a general indication of whether there are issues with measuring surface temperature, surface gravity, and check if any elements have `GRIDEDGE_BAD` set
- `ELEMFLAG`
    - Any example of this is `FE_H_FLAG`
        - Make sure that the flag does not contain `GRIDEDGE_BAD`, `GRIDEDGE_WARN` and `CALRANGE_BAD`
    - We should just check that the flag is equal to zero to eliminate all and any problematic behaviour

In [4]:
# Filter the data
STAR_BAD = 1 << 23
apogee = apogee.loc[apogee['ASPCAPFLAG'] & STAR_BAD == 0]
apogee = apogee.loc[apogee['FE_H_FLAG'] == 0]
apogee = apogee.loc[apogee['C_FE_FLAG'] == 0]
apogee = apogee.loc[apogee['AL_FE_FLAG'] == 0]
apogee = apogee.loc[apogee['N_FE_FLAG'] == 0]
apogee = apogee.loc[apogee['O_FE_FLAG'] == 0]
apogee = apogee.dropna()


Extract the useful column and save for later

In [ ]:
apogee = apogee[["APOGEE_ID","TEFF", "LOGG", "FE_H", "AL_FE", "C_FE", "N_FE", "O_FE"]]
apogee.columns = apogee.columns.str.lower()
apogee.to_csv("ProcessedData/cleaned_apogee.csv", index=False)

# Cleaning APOKSAC 2 Data



In [14]:
apoksac_cols = ["kic", "tmass_id", "mass"]
apoksac_col_width = [(0, 9), (9, 26), (27, 39)]

apoksac = pd.read_fwf(
    "RawData/asu.tsv",
    skiprows=42,
    header=None,
    index_col=False,
    names = apoksac_cols,
    colspecs = apoksac_col_width
)
apoksac = apoksac[0:6676]
apoksac = apoksac.dropna()


In [15]:
apoksac.to_csv("ProcessedData/cleaned_apoksac.csv", index=False)

Match APOKSAC data with APOGEE data

In [16]:
apogee = pd.read_csv("ProcessedData/cleaned_apogee.csv")
apogee = apogee[1:len(apogee)]

In [17]:
# Manipulate APOGEE id to 2MASS id
apogee = apogee.rename(columns = {"apogee_id": "tmass_id"})
apogee.tmass_id = apogee.tmass_id.str[4:-1]
apogee.to_csv("ProcessedData/cleaned_apogee.csv", index=False)

Now that we have separated the APOGEE and APOKSAC data, we need to merge them and save.

In [18]:
apoksac = pd.read_csv("ProcessedData/cleaned_apoksac.csv")
apogee = pd.read_csv("ProcessedData/cleaned_apogee.csv")

In [19]:
apoksac_mass_prediction_data = pd.merge(apogee, apoksac, on="tmass_id")
apoksac_mass_prediction_data.to_csv("ProcessedData/mass_prediction_apoksac.csv", index=False)

# Cleaning K2 Data
To do so, we need to

1. extract the mass coefficient from the data set
2. find the effective temperature by finding the data in the GALAH data set by matching the EPIC identifier
3. determine the mass
4. extract the chemical composition of stars
5. store the data for use

For the purpose of this, we will keep the mass of stars as a multiple of solar mass.

In [127]:
k2_cols = ["EPIC", "kappa-m-c"]
k2_col_width = [(0, 9), (24, 29)]

k2 = pd.read_fwf(
    "RawData/k2_mass_coefficients.txt",
    skiprows=50,
    header=None,
    index_col=False,
    names = k2_cols,
    colspecs = k2_col_width
)
# According to the International Astronomical Union (IAU), effective temperature of the sun is 5772 +- 0.8K
teff_sun = 5772
k2

,EPIC,kappa-m-c
0,201051625,1.324
1,201079541,1.523
2,201082897,1.027
3,201084449,1.544
4,201084664,1.076
...,...,...
18817,251619708,0.880
18818,251620873,0.663
18819,251621333,1.193
18820,251622742,1.497


Using Astroquery to create the link between EPIC id and GALAH's 2MASS id and source_id.

In [70]:
gaiadr3_table = Gaia.load_table('gaiadr3.gaia_source')

Retrieving table 'gaiadr3.gaia_source'


In [84]:
print(gaiadr3_table)
for column in gaiadr3_table.columns:
    print(column.name)

TAP Table name: gaiadr3.gaiadr3.gaia_source
Description: This table has an entry for every Gaia observed source as published with this data release. It contains the basic source parameters, in their final state as processed by the Gaia Data Processing and Analysis Consortium from the raw data coming from the spacecraft. The table is complemented with others containing information specific to certain kinds of objects (e.g.~Solar--system objects, non--single stars, variables etc.) and value--added processing (e.g.~astrophysical parameters etc.). Further array data types (spectra, epoch measurements) are presented separately via Datalink resources.
Num. columns: 152
solution_id
designation
source_id
random_index
ref_epoch
ra
ra_error
dec
dec_error
parallax
parallax_error
parallax_over_error
pm
pmra
pmra_error
pmdec
pmdec_error
ra_dec_corr
ra_parallax_corr
ra_pmra_corr
ra_pmdec_corr
dec_parallax_corr
dec_pmra_corr
dec_pmdec_corr
parallax_pmra_corr
parallax_pmdec_corr
pmra_pmdec_corr
astrom

In [125]:
gaia.MAIN_GAIA_TABLE = "gaiaedr3.gaia_source"
gaia = TapPlus(url="https://gea.esac.esa.int/tap-server/tap")

job = gaia.launch_job("select top 730000 \
    solution_id, source_id, designation \
    from gaiadr3.gaia_source order by source_id")
r = job.get_results()

In [126]:
r = r.to_pandas()
print(r)

                solution_id          source_id                 DESIGNATION
0       1636148068921376768         4295806720         Gaia DR3 4295806720
1       1636148068921376768        34361129088        Gaia DR3 34361129088
2       1636148068921376768        38655544960        Gaia DR3 38655544960
3       1636148068921376768       309238066432       Gaia DR3 309238066432
4       1636148068921376768       343597448960       Gaia DR3 343597448960
...                     ...                ...                         ...
729995  1636148068921376768  36153007473736320  Gaia DR3 36153007473736320
729996  1636148068921376768  36153110552919808  Gaia DR3 36153110552919808
729997  1636148068921376768  36153183569299840  Gaia DR3 36153183569299840
729998  1636148068921376768  36153213632154880  Gaia DR3 36153213632154880
729999  1636148068921376768  36153217926627456  Gaia DR3 36153217926627456

[730000 rows x 3 columns]


To determine the mass for K2 stars, we use the following formula (from K2 Galactic Archaeology Program DR3):

$$ \frac{M}{M_\odot} = \kappa_M (\frac{T_{eff}}{T_{eff, \odot}})^{3/2} $$

To do so, we will use [gaia-kepler.fun](https://gaia-kepler.fun) in order to determine the link K2 and GALAH data
- More information of the tables in the data set is given by [this documentation](https://exoplanetarchive.ipac.caltech.edu/docs/API_keplerstellar_columns.html)
- To do this, we will first match the K2 data with the `k2ToGaia`



In [22]:
k2ToGaia = Table.read("RawData/kepler_dr3_good.fits", format = "fits")
names = [name for name in k2ToGaia.colnames if len(k2ToGaia[name].shape) <= 1]
k2ToGaia = k2ToGaia[names].to_pandas()

In [23]:
k2ToGaia.columns

Index(['kepid', 'ra_kic', 'dec_kic', 'source_id', 'random_index', 'ra', 'dec',
       'parallax', 'parallax_error', 'parallax_over_error', 'pmra',
       'pmra_error', 'pmdec', 'pmdec_error', 'astrometric_params_solved',
       'pseudocolour', 'pseudocolour_error', 'ipd_frac_multi_peak',
       'ipd_frac_odd_win', 'ruwe', 'phot_g_mean_flux_over_error',
       'phot_g_mean_mag', 'phot_bp_mean_flux_over_error', 'phot_bp_mean_mag',
       'phot_rp_mean_flux_over_error', 'phot_rp_mean_mag',
       'phot_bp_rp_excess_factor', 'bp_rp', 'radial_velocity',
       'radial_velocity_error', 'rv_nb_transits', 'rv_expected_sig_to_noise',
       'rv_renormalised_gof', 'rv_chisq_pvalue', 'phot_variable_flag', 'l',
       'b', 'in_qso_candidates', 'in_galaxy_candidates', 'non_single_star',
       'has_xp_continuous', 'has_xp_sampled', 'has_rvs',
       'has_epoch_photometry', 'has_epoch_rv', 'has_mcmc_gspphot',
       'has_mcmc_msc', 'in_andromeda_survey', 'teff_gspphot', 'logg_gspphot',
       'mh_gs

In [24]:
k2ToGaia = k2ToGaia[["tm_designation", "kepid", "source_id"]]
k2ToGaia

,tm_designation,kepid,source_id
0,b'2MASS J19240929+3635532',757076,2050233803028318080
1,b'2MASS J19241033+3635377',757099,2050233601176543104
2,b'2MASS J19241341+3633358',757137,2050230543159814656
3,b'2MASS J19242289+3633538',757280,2050230611879323904
4,b'2MASS J19243302+3634385',757450,2050231848829944320
...,...,...,...
196757,b'2MASS J19201916+5227134',12984227,2139330189463960320
196758,b'2MASS J19202700+5224561',12984288,2139329875927860736
196759,b'2MASS J19202959+5228286',12984307,2139330636140575616
196760,b'2MASS J19204135+5225258',12984404,2139329467909467008


In [25]:
galah = pd.read_csv("ProcessedData/cleaned_galah.csv")
galah

,tmass_id,gaiadr3_source_id,teff,logg,fe_h,al_fe,c_fe,n_fe,o_fe
0,b'05225435-5857055',4762753422822077184,4747.2780,2.502026,-0.295569,0.242639,0.114089,0.048963,0.213739
1,b'05242161-5901319',4762739782006427648,5007.8013,4.504993,0.065576,0.186914,-0.039235,0.194514,-0.380775
2,b'05221820-5902598',4762730882833911808,4801.3870,4.519791,-0.054487,-0.114537,-0.167340,-0.317684,-0.461599
3,b'05215205-5900418',4762731226431207168,4970.5674,3.456641,-0.314558,-0.047509,0.278630,0.263258,0.441423
4,b'05222284-5903420',4762727927896411392,5278.6343,4.480108,0.221903,0.212420,0.019690,0.207750,-0.092077
...,...,...,...,...,...,...,...,...,...
72116,b'08460664-4312461',5524054930300489088,4604.0527,2.628620,-0.104896,0.592156,0.581064,0.267382,0.860856
72117,b'08431567-4230079',5524898599314763136,4921.5170,3.204505,0.015238,0.146001,0.029876,0.221036,0.257410
72118,b'08433930-4230037',5524851114156313600,4544.5430,2.545092,0.151254,0.061476,0.121679,-0.072001,0.024033
72119,b'08444698-4217078',5524953604957635456,4600.7554,4.267867,0.027199,-0.284221,0.148993,0.315737,0.339467


KeyError: 0